In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D
import h5py
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.callbacks import TensorBoard
import time
import random
import os
from sklearn.model_selection import train_test_split
import pickle 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install unrar
!unrar x "/content/drive/MyDrive/Shared Datasets/imgs and lbls.rar"

Streaming output truncated to the last 5000 lines.
Extracting  imgs and lbls/5328_0476911826_02_WRI-R1_F014.png              87%  OK 
Extracting  imgs and lbls/5328_0476911826_02_WRI-R1_F014.txt              87%  OK 
Extracting  imgs and lbls/5328_0476911872_02_WRI-R2_F014.png              87%  OK 
Extracting  imgs and lbls/5328_0476911872_02_WRI-R2_F014.txt              87%  OK 
Extracting  imgs and lbls/5329_0558345445_01_WRI-L1_F004.png              87%  OK 
Extracting  imgs and lbls/5329_0558345445_01_WRI-L1_F004.txt              87%  OK 
Extracting  imgs and lbls/5329_0558345475_01_WRI-L2_F004.png              87%  OK 
Extracting  imgs and lbls/5329_0558345475_01_WRI-L2_F004.txt              87%  OK 
Extracting  imgs and lbls/5329_0587343672_02_WRI-L1_F005.png              87%  OK 
Extracting  imgs and lbls/5329_0587343672_02_WRI-L1_F005.txt              87%  OK 
Extracting  imgs and lbls/532

In [ ]:
!unrar x "/content/drive/MyDrive/Shared Datasets/normal_augmented.rar"

Streaming output truncated to the last 5000 lines.
Extracting  normal_augmented/2611_0387448135_01_WRI-R2_F011-2.png         47%  OK 
Extracting  normal_augmented/2612_0948767038_01_WRI-L1_M012-1.png         47%  OK 
Extracting  normal_augmented/2612_0948767038_01_WRI-L1_M012-2.png         47%  OK 
Extracting  normal_augmented/2613_0873194547_02_WRI-L1_F007-1.png         47%  OK 
Extracting  normal_augmented/2613_0873194547_02_WRI-L1_F007-2.png         47%  OK 
Extracting  normal_augmented/2614_1008255987_01_WRI-L1_M007-1.png         47%  OK 
Extracting  normal_augmented/2614_1008255987_01_WRI-L1_M007-2.png         47%  OK 
Extracting  normal_augmented/2614_1008256044_02_WRI-L2_M007-1.png         47%  OK 
Extracting  normal_augmented/2614_1008256044_02_WRI-L2_M007-2.png         47%  OK 
Extracting  normal_augmented/2622_0694722474_01_WRI-L1_F012-1.png         48%  OK 
Extracting  normal_augmented/

In [3]:
CATEGORIES = [
    'boneanomaly', 'bonelesion', 'foreignbody', 'fracture', 
    'metal', 'periostealreaction', 'pronatorsign', 'softtissue', 'normal'
]

def encode_labels(values):
  multi_label = [0 for i in range(len(CATEGORIES))]
  for val in values:
    multi_label[val] = 1
  return multi_label

def get_multilabel_names(encoded_values):
  class_names = []
  for idx, val in enumerate(encoded_values):
    if val == 1:
      class_names.append(CATEGORIES[idx])
  return class_names

In [ ]:
import os
img_label_filename = []

HEIGHT = 150
WIDTH = 80

for dirname, dirs, files in os.walk('/content/imgs and lbls'):
  for file in tqdm(files):
  
    if '.txt' in file:
      classes = []

      with open(os.path.join(dirname, file), 'rb') as rbytes:
        line = 'init'
        while len(line) > 0:
          line = rbytes.readline()
          val = str(line).split(' ')[0][2]
          if val != "'" and val != '8':
            val = int(val)
            classes.append(val)
        if len(classes) == 0:
          classes.append(8)

      name = '{}'.format(file[:file.find('.txt')])
      img_name = '{}.png'.format(name)
      img_array = cv2.imread(os.path.join(dirname, img_name), cv2.IMREAD_GRAYSCALE)
      img_array = cv2.resize(img_array, (WIDTH, HEIGHT))
      img_label_filename.append([img_array, encode_labels(classes), name])

100%|██████████| 40655/40655 [04:37<00:00, 146.38it/s]


In [ ]:
for file in tqdm(os.listdir('/content/normal_augmented')):
  name = '{}'.format(file[:file.find('.png')])
  img_name = '{}.png'.format(name)
  img_array = cv2.imread(os.path.join('/content/normal_augmented', img_name), cv2.IMREAD_GRAYSCALE)
  img_array = cv2.resize(img_array, (WIDTH, HEIGHT))
  img_label_filename.append([img_array, encode_labels([8]), name])

100%|██████████| 9664/9664 [01:05<00:00, 146.68it/s]


In [ ]:
len(img_label_filename)

29991

In [ ]:
img_label_filename[29990]

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 '2673_0725669898_01_WRI-L1_F009-1']

In [ ]:
random.shuffle(img_label_filename)

testing = img_label_filename[:5999]
validation = img_label_filename[5999:9000]
training = img_label_filename[9000:]

In [ ]:
img_label_filename = []

In [ ]:
print(f'Training size: {len(training)}, Testing size: {len(testing)}, Validation size: {len(validation)}')

Training size: 20991, Testing size: 5999, Validation size: 3001


In [ ]:
train_data = []
train_labels = []

for img, label, filename in tqdm(training):
  train_data.append(img)
  train_labels.append(label)

test_data = []
test_labels = []

for img, label, filename in tqdm(testing):
  test_data.append(img)
  test_labels.append(label)


val_data = []
val_labels = []

for img, label, filename in tqdm(validation):
  val_data.append(img)
  val_labels.append(label)

100%|██████████| 3001/3001 [00:00<00:00, 1048314.01it/s]


In [ ]:
training = []
testing = []
validation = []

In [ ]:
train_data = np.array(train_data).reshape(-1, HEIGHT, WIDTH, 1)
train_data = np.array(train_data).astype('float')/255.0

test_data = np.array(test_data).reshape(-1, HEIGHT, WIDTH, 1)
test_data = np.array(test_data).astype('float')/255.0

val_data = np.array(val_data).reshape(-1, HEIGHT, WIDTH, 1)
val_data = np.array(val_data).astype('float')/255.0

In [ ]:
pickle_out = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/train_data(aug).pickle', 'wb')
pickle.dump(train_data, pickle_out)
pickle_out.close()

pickle_out = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/test_data(aug).pickle', 'wb')
pickle.dump(test_data, pickle_out)
pickle_out.close()

pickle_out = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/val_data(aug).pickle', 'wb')
pickle.dump(val_data, pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/train_labels(aug).pickle', 'wb')
pickle.dump(train_labels, pickle_out)
pickle_out.close()

pickle_out = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/test_labels(aug).pickle', 'wb')
pickle.dump(test_labels, pickle_out)
pickle_out.close()

pickle_out = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/val_labels(aug).pickle', 'wb')
pickle.dump(val_labels, pickle_out)
pickle_out.close()
# pickle_out = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/filenames(aug).pickle', 'wb')
# pickle.dump(filenames, pickle_out)
# pickle_out.close()

In [ ]:
# Load imgs 
pickle_in = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/train_data(aug).pickle', 'rb')
train_data = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/test_data(aug).pickle', 'rb')
test_data = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/val_data(aug).pickle', 'rb')
val_data = pickle.load(pickle_in)
pickle_in.close()

# pickle_in = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/filenames(aug).pickle', 'rb')
# filenames = pickle.load(pickle_in)
# pickle_in.close()

In [ ]:
# Load labels
pickle_in = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/train_labels(aug).pickle', 'rb')
train_labels = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/test_labels(aug).pickle', 'rb')
test_labels = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/val_labels(aug).pickle', 'rb')
val_labels = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
print(len(train_data), len(train_labels))
print(len(test_data), len(test_labels))
print(len(val_data), len(val_labels))

20991 20991
5999 5999
3001 3001


In [ ]:
train_labels_df = pd.DataFrame(columns=CATEGORIES)
test_labels_df = pd.DataFrame(columns=CATEGORIES)
val_labels_df = pd.DataFrame(columns=CATEGORIES)

for label in tqdm(train_labels):
  train_labels_df.loc[len(train_labels_df)] = label

for label in tqdm(test_labels):
  test_labels_df.loc[len(test_labels_df)] = label

for label in tqdm(val_labels):
  val_labels_df.loc[len(val_labels_df)] = label

100%|██████████| 3001/3001 [00:06<00:00, 475.64it/s]


In [ ]:
train_labels = np.array(train_labels_df)
test_labels = np.array(test_labels_df)
val_labels = np.array(val_labels_df)

In [ ]:
pickle_out = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/train_labels_df(aug).pickle', 'wb')
pickle.dump(train_labels, pickle_out)
pickle_out.close()

pickle_out = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/test_labels_df(aug).pickle', 'wb')
pickle.dump(test_labels, pickle_out)
pickle_out.close()

pickle_out = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/val_labels_df(aug).pickle', 'wb')
pickle.dump(val_labels, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/train_labels_df(aug).pickle', 'rb')
train_labels = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/test_labels_df(aug).pickle', 'rb')
test_labels = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('/content/drive/MyDrive/Shared Datasets/Loaded data (multilabel)/val_labels_df(aug).pickle', 'rb')
val_labels = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
print(len(train_labels), len(test_labels), len(val_labels))

20991 5999 3001


In [ ]:
train_labels[0]

array([0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=object)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, random_state=0, test_size=0.2)

In [ ]:
imgs = []
labels = []
# label_df = []
filenames = []

In [ ]:
# X_val, y_val = X_train[:2064], y_train[:2064]
# X_train, y_train = X_train[2064:], y_train[2064:]


In [ ]:
train_data = np.asarray(train_data).astype(np.float32)
test_data = np.asarray(test_data).astype(np.float32)
val_data = np.asarray(val_data).astype(np.float32)

In [ ]:
train_labels = np.asarray(train_labels).astype(np.float32)
test_labels = np.asarray(test_labels).astype(np.float32)
val_labels = np.asarray(val_labels).astype(np.float32)

In [ ]:
dense_layers = [0, 1]
layer_sizes = [16, 32]
conv_layers = [1, 2, 3, 4]

models = []

for dense_layer in dense_layers:
	for layer_size in layer_sizes:
		for conv_layer in conv_layers:
			NAME = f'wri-{conv_layer}-conv-{layer_size}-layer-{dense_layer}-dense-{int(time.time())}'
			tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
			print(NAME)
			# %load_ext tensorboard
			# %tensorboard --logdir logs
			model = Sequential()
			model.add(Conv2D(layer_size, (3, 3), input_shape=(150, 80, 1), activation='relu'))
			model.add(BatchNormalization())
			model.add(MaxPooling2D(pool_size=(2, 2)))
			model.add(Dropout(0.3))
	 
			for i in range(conv_layer - 1):
				model.add(Conv2D(layer_size, (3, 3), activation='relu'))
				model.add(MaxPooling2D(pool_size=(2, 2)))
				model.add(Dropout(0.3))
		
			model.add(Flatten())
			for l in range(dense_layer):
				model.add(Dense(512, activation='relu'))
				model.add(Dropout(0.5))

			model.add(Dense(9, activation='sigmoid'))
	 
			model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
			model.fit(train_data, train_labels, validation_data=(val_data, val_labels), batch_size=100, epochs=5, callbacks=[tensorboard])
			val_loss, val_acc = model.evaluate(val_data, val_labels)
			models.append([f'{NAME}, LOSS: {val_loss}, ACC: {val_acc}', model])
			# models.append(f'{NAME}, LOSS: {val_loss}, ACC: {val_acc}')

wri-1-conv-16-layer-0-dense-1663594676
Epoch 1/5
210/210 [==============================] - 16s 18ms/step - loss: 0.1981 - accuracy: 0.7348 - val_loss: 0.2243 - val_accuracy: 0.5348
Epoch 2/5
210/210 [==============================] - 3s 16ms/step - loss: 0.1383 - accuracy: 0.7906 - val_loss: 0.1924 - val_accuracy: 0.5988
Epoch 3/5
210/210 [==============================] - 3s 16ms/step - loss: 0.1204 - accuracy: 0.8100 - val_loss: 0.1614 - val_accuracy: 0.7331
Epoch 4/5
210/210 [==============================] - 3s 16ms/step - loss: 0.1064 - accuracy: 0.8236 - val_loss: 0.1681 - val_accuracy: 0.7354
Epoch 5/5
94/94 [==============================] - 0s 4ms/step - loss: 0.1752 - accuracy: 0.7637
wri-2-conv-16-layer-0-dense-1663594723
Epoch 1/5
210/210 [==============================] - 5s 21ms/step - loss: 0.1840 - accuracy: 0.7370 - val_loss: 0.5389 - val_accuracy: 0.5515
Epoch 2/5
210/210 [==============================] - 4s 19ms/step - loss: 0.1462 - accuracy: 0.7784 - val_loss: 0.

In [ ]:
models

[['wri-1-conv-16-layer-0-dense-1663594676, LOSS: 0.17519551515579224, ACC: 0.7637454271316528',
 ['wri-2-conv-16-layer-0-dense-1663594723, LOSS: 0.1415475606918335, ACC: 0.7844051718711853',
 ['wri-3-conv-16-layer-0-dense-1663594745, LOSS: 0.15065516531467438, ACC: 0.7674108743667603',
 ['wri-4-conv-16-layer-0-dense-1663594769, LOSS: 0.15040796995162964, ACC: 0.7940686345100403',
 ['wri-1-conv-32-layer-0-dense-1663594792, LOSS: 0.1662152260541916, ACC: 0.7850716710090637',
 ['wri-2-conv-32-layer-0-dense-1663594822, LOSS: 0.13385482132434845, ACC: 0.7964012026786804',
 ['wri-3-conv-32-layer-0-dense-1663594860, LOSS: 0.16332083940505981, ACC: 0.718093991279602',
 ['wri-4-conv-32-layer-0-dense-1663594899, LOSS: 0.1528511643409729, ACC: 0.7700766324996948',
 ['wri-1-conv-16-layer-1-dense-1663594938, LOSS: 0.15039873123168945, ACC: 0.7960679531097412',
 ['wri-2-conv-16-layer-1-dense-1663594964, LOSS: 0.15386314690113068, ACC: 0.7507497668266296',
 ['wri-3-conv-16-layer-1-dense-1663594988, L

In [ ]:
dense_layers = [1]
layer_sizes = [64, 128]
conv_layers = [1, 2, 3, 4]

for dense_layer in dense_layers:
	for layer_size in layer_sizes:
		for conv_layer in conv_layers:
			NAME = f'wri-{conv_layer}-conv-{layer_size}-layer-{dense_layer}-dense-{int(time.time())}'
			tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
			print(NAME)
			# %load_ext tensorboard
			# %tensorboard --logdir logs
			model = Sequential()
			model.add(Conv2D(layer_size, (3, 3), input_shape=(150, 80, 1), activation='relu'))
			model.add(BatchNormalization())
			model.add(MaxPooling2D(pool_size=(2, 2)))
			model.add(Dropout(0.3))
	 
			for i in range(conv_layer - 1):
				model.add(Conv2D(layer_size, (3, 3), activation='relu'))
				model.add(MaxPooling2D(pool_size=(2, 2)))
				model.add(Dropout(0.3))
		
			model.add(Flatten())
			for l in range(dense_layer):
				model.add(Dense(512, activation='relu'))
				model.add(Dropout(0.5))

			model.add(Dense(9, activation='sigmoid'))
	 
			model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
			model.fit(train_data, train_labels, validation_data=(val_data, val_labels), batch_size=100, epochs=5, callbacks=[tensorboard])
			val_loss, val_acc = model.evaluate(val_data, val_labels)
			models.append([f'{NAME}, LOSS: {val_loss}, ACC: {val_acc}', model])
			# models.append(f'{NAME}, LOSS: {val_loss}, ACC: {val_acc}')

wri-1-conv-64-layer-1-dense-1663595310
Epoch 1/5
210/210 [==============================] - 16s 76ms/step - loss: 0.5283 - accuracy: 0.7162 - val_loss: 0.2215 - val_accuracy: 0.5378
Epoch 2/5
210/210 [==============================] - 16s 75ms/step - loss: 0.1482 - accuracy: 0.7834 - val_loss: 0.1793 - val_accuracy: 0.7188
Epoch 3/5
210/210 [==============================] - 16s 75ms/step - loss: 0.1288 - accuracy: 0.8151 - val_loss: 0.1896 - val_accuracy: 0.7241
Epoch 4/5
210/210 [==============================] - 16s 75ms/step - loss: 0.1157 - accuracy: 0.8375 - val_loss: 0.1417 - val_accuracy: 0.8081
Epoch 5/5
94/94 [==============================] - 1s 8ms/step - loss: 0.2138 - accuracy: 0.7491
wri-2-conv-64-layer-1-dense-1663595392
Epoch 1/5


ResourceExhaustedError: ignored

In [ ]:

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.999)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True, gpu_options=gpu_options))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 80, 1), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
# model.add(Dense(512, activation='relu'))
model.add(Dense(9, activation='sigmoid'))

model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), batch_size=100, epochs=18)

Epoch 1/18
210/210 [==============================] - 9s 40ms/step - loss: 0.1855 - accuracy: 0.6979 - val_loss: 0.1556 - val_accuracy: 0.7637
Epoch 2/18
210/210 [==============================] - 8s 39ms/step - loss: 0.1503 - accuracy: 0.7676 - val_loss: 0.1350 - val_accuracy: 0.7931
Epoch 3/18
210/210 [==============================] - 8s 39ms/step - loss: 0.1369 - accuracy: 0.7881 - val_loss: 0.1264 - val_accuracy: 0.8061
Epoch 4/18
210/210 [==============================] - 8s 39ms/step - loss: 0.1288 - accuracy: 0.8002 - val_loss: 0.1231 - val_accuracy: 0.7977
Epoch 5/18
210/210 [==============================] - 8s 39ms/step - loss: 0.1243 - accuracy: 0.8049 - val_loss: 0.1186 - val_accuracy: 0.8077
Epoch 6/18
210/210 [==============================] - 8s 39ms/step - loss: 0.1216 - accuracy: 0.8077 - val_loss: 0.1210 - val_accuracy: 0.8087
Epoch 7/18
210/210 [==============================] - 8s 39ms/step - loss: 0.1177 - accuracy: 0.8165 - val_loss: 0.1112 - val_accuracy: 0.8274

In [4]:
def save_model(model_name, model):
    model_json = model.to_json()
    with open(f"{model_name}.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(f"{model_name}.h5")
    print("Saved model to disk")

def load_model(model_json, model_weights, model_name):
    json_file = open(model_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    conv_model = model_from_json(loaded_model_json)
    # load weights into new model
    conv_model.load_weights(model_weights)
    print("Loaded model from disk")
    return conv_model, model_name

In [ ]:
save_model('15-epoch-84.71-dropout-model', model)

Saved model to disk


In [ ]:
performances = []

In [ ]:
model, name = load_model('/content/drive/MyDrive/Shared Datasets/Saved Models/aug_multi_label/15-epoch-84.71-dropout-model.json', '/content/drive/MyDrive/Shared Datasets/Saved Models/aug_multi_label/15-epoch-84.71-dropout-model.h5', '15-epoch-84.71-dropout-model')
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
test_loss, test_acc = model.evaluate(test_data, test_labels)
train_loss, train_acc = model.evaluate(train_data, train_labels)
performances.append(f'{name}, test_loss: {test_loss}, test_acc: {test_acc}, train_acc: {train_acc}')

Loaded model from disk
656/656 [==============================] - 5s 7ms/step - loss: 0.0594 - accuracy: 0.9098


NameError: ignored

In [5]:
model, name = load_model('/content/drive/MyDrive/Shared Datasets/Saved Models/aug_multi_label/15-epoch-84.71-dropout-model.json', '/content/drive/MyDrive/Shared Datasets/Saved Models/aug_multi_label/15-epoch-84.71-dropout-model.h5', '15-epoch-84.71-dropout-model')
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

Loaded model from disk


In [ ]:
performances

['134page-15-epoch-model, test_loss: 0.12217538058757782, test_acc: 0.8423070311546326, train_acc: 0.9302082061767578',
 '134page-config-model, test_loss: 0.11156488209962845, test_acc: 0.8353058695793152, train_acc: 0.8723738789558411',
 '15-epoch-84.71-dropout-model, test_loss: 0.1038149818778038, test_acc: 0.8564760684967041, train_acc: 0.9098184704780579',
 '15-epoch-dropout-model, test_loss: 0.10354877263307571, test_acc: 0.8426404595375061, train_acc: 0.8805678486824036']

In [6]:
def load_img(img_path):
  img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  img_array = cv2.resize(img_array, (80, 150))
  img_array = np.array(img_array).reshape(-1, 150, 80, 1)
  img_array = (np.array(img_array)).astype('float')/255.0
  return img_array

In [12]:
pred = model.predict(load_img('/content/sample7.PNG'))
sorted_categories = np.argsort(pred[0])[:-11:-1]

In [13]:
sorted_categories
for i in range(9):
    print("{}".format(CATEGORIES[sorted_categories[i]])+" ({:.3})".format(pred[0][sorted_categories[i]]))

fracture (0.702)
normal (0.315)
periostealreaction (0.0888)
bonelesion (0.013)
boneanomaly (0.0063)
softtissue (0.00371)
foreignbody (0.000781)
pronatorsign (0.00029)
metal (0.000153)
